In [1]:
# Import
from urllib.request import urlopen
from bs4 import BeautifulSoup as bs
from datetime import datetime
import os
import numpy as np
import json
import pandas as pd
import requests
import openpyxl

# Variables
url = "https://www.fangraphs.com/api/depth-charts/roster?teamid="
team_ids = np.arange(1,31)
data = pd.DataFrame()
teamlist = [
    [1,'Los Angeles Angels'],
    [2,'Baltimore Orioles'],
    [3,'Boston Red Sox'],
    [4,'Chicago White Sox'],
    [5,'Cleveland Indians'],
    [6,'Detroit Tigers'],
    [7,'Kansas City Royals'],
    [8,'Minnesota Twins'],
    [9,'New York Yankees'],
    [10,'Oakland Athletics'],
    [11,'Seattle Mariners'],
    [12,'Tampa Bay Rays'],
    [13,'Texas Rangers'],
    [14,'Toronto Blue Jays'],
    [15,'Arizona Diamondbacks'],
    [16,'Atlanta Braves'],
    [17,'Chicago Cubs'],
    [18,'Cincinnatti Reds'],
    [19,'Colorado Rockies'],
    [20,'Miami Marlins'],
    [21,'Houston Astros'],
    [22,'Los Angeles Dodgers'],
    [23,'Milwaukee Brewers'],
    [24,'Washington Nationals'],
    [25,'New York Mets'],
    [26,'Philadelphia Phillies'],
    [27,'Pittsburgh Pirates'],
    [28,'St. Louis Cardinals'],
    [29,'San Diego Padres'],
    [30,'San Francisco Giants']
]
teamlist_df = pd.DataFrame(teamlist, columns=['teamid', 'teamname'])
current_year = '2024'
ffs_database = pd.read_csv('input files/FFS Database - Database.csv', encoding = "ISO-8859-1")
ffs_database['Name'] = ffs_database['Name'].str.lower().str.strip()

# Pull data for each team
for team_id in team_ids:
    
    team_url = f"{url}{team_id}"
    df = pd.read_json(team_url)
    data = pd.concat([data, df])
    #data = data.append(df)

# Format player name text
data['player'] = data['player'].str.replace('.','', regex=False) #This is all of the depth charts combined
data['player'] = data['player'].str.replace('ñ', 'n', regex=False)
data['player'] = data['player'].str.replace('’', "'", regex=False)
data['player'] = data['player'].str.replace('Jr', '', regex=False)
data['player'] = data['player'].str.lower().str.strip()
data = pd.merge(data, teamlist_df, on='teamid', how='outer')

# Find all 26 man
active_roster = data[data['type'].str.contains('mlb-').fillna(False)]

# Find all 40 man
forty_man = data[data['roster40'].str.contains('Y').fillna(False)]

ar40 = pd.merge(active_roster, forty_man, how='outer')



C:\Users\root\AppData\Local\Temp\ipykernel_5200\2552838820.py:50: DtypeWarning: Columns (7,9,11,12,13,14,15,16,17,18,19,20,21,22,23,37,38,41,42,62) have mixed types. Specify dtype option on import or set low_memory=False.
  ffs_database = pd.read_csv('input files/FFS Database - Database.csv', encoding = "ISO-8859-1")


In [2]:
# Import FFS Rosters
# Variables
file_name = "FFS Spreadsheet.xlsx"
ffs_teams_list = ['ARI','ATL','BAL','BOS','CHC','CHW','CIN','CLE','COL','DET','HOU','KC','LAA','LAD','MIA','MIL','MIN','NYM','NYY','OAK','PHI','PIT','SD','SEA','SF','STL','TB','TEX','TOR','WAS']
name_cols = "G"
change_list = []
sheet_data = []

for team in ffs_teams_list:
    # Variables
    sheet = pd.read_excel(file_name, sheet_name=team, usecols=name_cols, skiprows=2, engine='openpyxl')
    sheet_names = sheet['Player']

    # Remove empty rows
    nan_value = float("NaN")
    sheet_names.replace("", nan_value, inplace=True)
    sheet_names.dropna(inplace=True)

    # Write progress to console
    #print(f"Processing data for {team}")

    # Loop through each name in the selected team sheet
    for sheet_name in sheet_names:

        # Remove FFS lingo from name (formatting)
        sheet_name = sheet_name.replace('RV', '')
        sheet_name = sheet_name.replace('R2', '')       
        sheet_name = sheet_name.replace('R3', '')          
        sheet_name = sheet_name.replace('R4', '')          
        sheet_name = sheet_name.replace('R5', '')          
        sheet_name = sheet_name.replace('R6', '')          
        sheet_name = sheet_name.replace('7TH', '')
        sheet_name = sheet_name.replace('8TH', '')
        sheet_name = sheet_name.replace('9TH', '')
        sheet_name = sheet_name.replace('10TH', '')
        sheet_name = sheet_name.replace('11TH', '')
        sheet_name = sheet_name.replace('R1', '')
        sheet_name = sheet_name.replace('’', "'")
        sheet_name = sheet_name.replace('.', '')
        sheet_name = sheet_name.replace('Jr', '')
        sheet_name = sheet_name.replace(team, '')
        sheet_name = sheet_name.lower()
        sheet_name = sheet_name.strip()
        
        sheet_data.append({
            "player": sheet_name,
            "owner": team
        })
        
ffs_data = pd.DataFrame(sheet_data)



In [3]:
# Find Unowned 40 man
matched = pd.merge(ar40, ffs_data, on='player', how='outer', indicator=True)
spell_check = matched[matched['_merge'] == 'left_only']
spell_check = spell_check.drop(['_merge'], axis=1)
spell_check['player'] = spell_check['player'].str.lower().str.strip()

# Find Rule 5 eligible
rule_v_matched_ffs = pd.merge(ffs_database, ffs_data, left_on = "Name", right_on = "player", how='outer', indicator=True)
rule_v_left = rule_v_matched_ffs[rule_v_matched_ffs['_merge'] == 'both']
rule_v_owned = rule_v_left[~rule_v_left['owner'].isna()]
rule_v_25 = rule_v_owned[rule_v_owned['Age'] > 24]
rule_v_eligible = rule_v_25[rule_v_25['40man'] != 'x']
rule_v_eligible = rule_v_eligible.drop(['_merge'], axis=1)
rule_v_eligible.to_csv (f"output files/rule_v.csv", index = False, header=True)

# Find Rule 5 on 40 man
rule_v_forty = pd.merge(ar40, rule_v_eligible, left_on = "player", right_on = "Name", how='inner', indicator=True)
rule_v_forty.to_csv (f"output files/rule_v_forty.csv", index = False, header=True)

# Import alias list
aliases = pd.read_csv('input files/aliases.csv', encoding = "ISO-8859-1").apply(lambda x: x.astype(str).str.lower())
alias_list = list(aliases.columns)

for alias in alias_list:
    if '_merge' in spell_check:
        spell_check = spell_check.drop(['_merge'], axis=1)
    spell_check = pd.merge(spell_check, aliases, left_on='player', right_on=alias, how='outer', indicator=True)
    spell_check = spell_check[spell_check['_merge'] == 'left_only']
    
spell_check['player'] = spell_check['player'].str.title()
spell_check.loc[:, spell_check.dtypes.eq(float)] = spell_check.select_dtypes(float).fillna(0)
spell_check.loc[:, 'servicetime1'] = spell_check['servicetime1'].replace('', 0.000).astype('float')
spell_check = spell_check.sort_values(by=['servicetime1'], ascending=[False]).reset_index(drop=True)

# Fangraphs IDs
minormasterid = spell_check['minormasterid']
spell_check['api_id'] = spell_check['playerid'].fillna(spell_check['minormasterid'])
spell_check['api_id'] = np.where(spell_check['api_id'] == 0, minormasterid, spell_check['api_id'])
fg_link = "http://www.fangraphs.com/statss.aspx?playerid=" + spell_check['api_id'].astype(str).apply(lambda x: x.replace('.0', ''))
fg_api_link = "https://www.fangraphs.com/api/players/stats?position=&playerid=" + spell_check['api_id'].astype(str).apply(lambda x: x.replace('.0', ''))
appeared = []
for link in fg_api_link:
    try:
        appeared_year = requests.get(link).json()['playerInfo']['LastGame'][:4]
    except:
        appeared_year = ''
    if appeared_year == current_year:
        appeared.append('Y')
    else:
        appeared.append('')
spell_check['appeared'] = appeared

# MLB/MiLB IDs
minorbamid = spell_check['minorbamid'].astype(int).astype(str)
spell_check['mlb_link_id'] = spell_check['mlbamid1'].fillna(spell_check['minorbamid'])
spell_check['mlb_link_id'] = np.where(spell_check['mlb_link_id'] == 0, minorbamid, spell_check['mlb_link_id'])
mlb_link = "http://www.mlb.com/player/" + spell_check['mlb_link_id'].astype(str).apply(lambda x: x.replace('.0', ''))

# Filter players
mlb_sl = spell_check[(spell_check['type'].str.contains('mlb-')) & (spell_check['roster40'] == 'Y')]
nri = spell_check[(spell_check['type'].str.contains('mlb-')) & (spell_check['roster40'] == 'N')]
forty = spell_check[(~spell_check['type'].str.contains('mlb-')) & (spell_check['roster40'] == 'Y')]



# Change names of yesterday's exports if they exist
# Export as csv for reference tomorrow (include FFS Sheet)
datetoday = datetime.now().strftime("%Y%m%d")

# if os.path.isfile(f"output files/new.csv"):
#     if os.path.isfile('output files/old.csv'):
#         os.rename (f"output files/old.csv", f"output files/{datetoday}.csv")
#     os.rename (f"output files/new.csv", f"output files/old.csv")
# else:
#     pass

spell_check.to_csv (f"output files/new.csv", encoding = "ISO-8859-1", index = False, header=True)

# Import yesterday's csvs
old_df = pd.read_csv('output files/old.csv', encoding = "ISO-8859-1")
new_df = pd.read_csv('output files/new.csv', encoding = "ISO-8859-1")
changes_df = pd.concat([old_df, new_df]).drop_duplicates(subset="type", keep=False)#.reset_index(drop=True)

# Compare changes and make lists for HTML



C:\Users\root\AppData\Local\Temp\ipykernel_5200\532451370.py:32: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  spell_check.loc[:, 'servicetime1'] = spell_check['servicetime1'].replace('', 0.000).astype('float')


In [4]:
# HTML
mlb_playerlist = '<tr>\
                <td style="color:black;" bgcolor="50ae26" align="center" border="1">' + mlb_sl['position'].astype(str) + '</td>\
                <td style="color:black;" bgcolor="50ae26" align="center">' + mlb_sl['player'].astype(str) + '</td>\
                <td style="color:black;" bgcolor="50ae26" align="center">' + mlb_sl['teamname'] + '</td>\
                <td style="color:black;" bgcolor="50ae26" align="center">' + mlb_sl['age'].astype(str) + '</td>\
                <td style="color:black;" bgcolor="50ae26" align="center">' + mlb_sl['servicetime1'].astype(str) + '</td>\
                <td style="color:black;" bgcolor="50ae26" align="center">' + mlb_sl['appeared'].astype(str) + '</td>\
                <td style="color:black;" bgcolor="50ae26" align="center"><a href="' + fg_link + '"><img src="https://i.ibb.co/BZwJGdb/fg.png" style="width:20px;height:20px;"></a></td>\
                <td style="color:black;" bgcolor="50ae26" align="center"><a href="' + mlb_link + '"><img src="https://i.ibb.co/j6mc9yH/mlb.jpg" style="width:20px;height:20px;"></a></td>\
                </tr>'
mlb_playerlist = mlb_playerlist.dropna()
mlb_html = ' '.join(mlb_playerlist)

nri_playerlist = '<tr>\
                <td style="color:black;" bgcolor="ffffff" align="center" border="1">' + nri['position'].astype(str) + '</td>\
                <td style="color:black;" bgcolor="ffffff" align="center">' + nri['player'].astype(str) + '</td>\
                <td style="color:black;" bgcolor="ffffff" align="center">' + nri['teamname'] + '</td>\
                <td style="color:black;" bgcolor="ffffff" align="center">' + nri['age'].astype(str) + '</td>\
                <td style="color:black;" bgcolor="ffffff" align="center">' + nri['servicetime1'].astype(str) + '</td>\
                <td style="color:black;" bgcolor="ffffff" align="center">' + nri['appeared'].astype(str) + '</td>\
                <td style="color:black;" bgcolor="ffffff" align="center"><a href="' + fg_link + '"><img src="https://i.ibb.co/BZwJGdb/fg.png" style="width:20px;height:20px;"></a></td>\
                <td style="color:black;" bgcolor="ffffff" align="center"><a href="' + mlb_link + '"><img src="https://i.ibb.co/j6mc9yH/mlb.jpg" style="width:20px;height:20px;"></a></td>\
                </tr>'
nri_playerlist = nri_playerlist.dropna()
nri_html = ' '.join(nri_playerlist)

forty_playerlist = '<tr>\
                <td style="color:black;" bgcolor="bd9048" align="center" border="1">' + forty['position'].astype(str) + '</td>\
                <td style="color:black;" bgcolor="bd9048" align="center">' + forty['player'].astype(str) + '</td>\
                <td style="color:black;" bgcolor="bd9048" align="center">' + forty['teamname'] + '</td>\
                <td style="color:black;" bgcolor="bd9048" align="center">' + forty['age'].astype(str) + '</td>\
                <td style="color:black;" bgcolor="bd9048" align="center">' + forty['servicetime1'].astype(str) + '</td>\
                <td style="color:black;" bgcolor="bd9048" align="center">' + forty['appeared'].astype(str) + '</td>\
                <td style="color:black;" bgcolor="bd9048" align="center"><a href="' + fg_link + '"><img src="https://i.ibb.co/BZwJGdb/fg.png" style="width:20px;height:20px;"></a></td>\
                <td style="color:black;" bgcolor="bd9048" align="center"><a href="' + mlb_link + '"><img src="https://i.ibb.co/j6mc9yH/mlb.jpg" style="width:20px;height:20px;"></a></td>\
                </tr>'
forty_playerlist = forty_playerlist.dropna()
forty_html = ' '.join(forty_playerlist)

# Send email
address_list = ["scottrehn@gmail.com", "Ryan.Smolarek@gmail.com"]

import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

def send_email(subject, text, html, sender, recipients, password):
    msg = MIMEMultipart('alternative')
    msg['Subject'] = subject
    msg['From'] = sender
    msg['To'] = ', '.join(recipients)
    
    # Record the MIME types of both parts - text/plain and text/html.
    part1 = MIMEText(text, 'plain')
    part2 = MIMEText(html, 'html')

    # Attach parts into message container.
    # According to RFC 2046, the last part of a multipart message, in this case
    # the HTML message, is best and preferred.
    msg.attach(part1)
    msg.attach(part2)
        
    smtp_server = smtplib.SMTP_SSL('smtp.gmail.com', 465)
    smtp_server.login(sender, password)
    smtp_server.sendmail(sender, recipients, msg.as_string())
    smtp_server.quit()

subject = "FFS Daily Digest"
# css = """\
# <style>
# .toggle        { display: none; }
# .toggle:target { display: table; }
# </style>"""
text = ""
change_notes = ""
html = f"""\
<html>
  <head></head>
  <body>
    <h3 align="center" style="color:black;">Protected MLB Players Unowned In FFS:</h3>
    <h5 align="center" style="color:black;">Change notes: <br> {change_notes}</h5>
      <div>
        <table border="1" align="center" cellpadding="5" cellspacing="2" >
          <th bgcolor="50ae26">Position</th>
          <th bgcolor="50ae26">Player</th>
          <th bgcolor="50ae26">Team</th>
          <th bgcolor="50ae26">Age</th>
          <th bgcolor="50ae26">Service Time</th>
          <th bgcolor="50ae26">Appeared</th>
          <th bgcolor="50ae26">FG</th>
          <th bgcolor="50ae26">MLB</th>
            <tbody>
              {mlb_html}
              {nri_html}
            </tbody>
        </table>
      </div>
      
      <br>
    
      <div>
        <table class="toggle" border="1" align="center" cellpadding="5" cellspacing="2" >
          <th bgcolor="bd9048">Position</th>
          <th bgcolor="bd9048">Player</th>
          <th bgcolor="bd9048">Team</th>
          <th bgcolor="bd9048">Age</th>
          <th bgcolor="bd9048">Service Time</th>
          <th bgcolor="bd9048">Appeared</th>
          <th bgcolor="bd9048">FG</th>
          <th bgcolor="bd9048">MLB</th>
            <tbody>
              {forty_html}
            </tbody>
        </table>
      </div>  
    
    <br>
    <table border="1" align="center" cellpadding="5" cellspacing="2">
      <tr>
        <td bgcolor="b1ff90" align="center" border="1">On MLB Active Roster</td>
      </tr>
      <tr>
        <td bgcolor="ffffff" align="center" border="1">NRI</td>
      </tr>
      <tr>
        <td bgcolor="bd9048" align="center" border="1">On MLB 40 Man Roster</td>
      </tr>
    </table>
  </body>
  <h5 align="center">
    This email will send every morning at 8am CST.
    <br>
    Suggestions welcome! Let me know what you think!
    <br>
    <br>
    <a href="mailto:reds.ffsmlb@gmail.com?subject=Unsubscribe me&body=Unsubscribe me">Unsubscribe</a>
  </h5>
</html>
"""
sender = "RedsFFSMLB@gmail.com"
recipients = address_list
password = "sargqsqqtyqzwlbw"

send_email(subject, text, html, sender, recipients, password)



In [5]:

# Change names of yesterday's exports if they exist
# Export as csv for reference tomorrow (include FFS Sheet)
datetoday = datetime.now().strftime("%Y%m%d")

# if os.path.isfile(f"output files/new.csv"):
#     if os.path.isfile('output files/old.csv'):
#         os.rename (f"output files/old.csv", f"output files/{datetoday}.csv")
#     os.rename (f"output files/new.csv", f"output files/old.csv")
# else:
#     pass

spell_check.to_csv (f"output files/new.csv", encoding = "ISO-8859-1", index = False, header=True)

# Import yesterday's csvs
old_df = pd.read_csv('output files/old.csv', encoding = "ISO-8859-1")
new_df = pd.read_csv('output files/new.csv', encoding = "ISO-8859-1")
changes_df = pd.concat([old_df, new_df])#.drop_duplicates(subset="type", keep=False)#.reset_index(drop=True)
changes_df

,teamid,loaddate,type,role,position,jnum,player,notes,handed,age,...,actualz_pit_BB%,actualz_pit_K%,actual_pit_pivFA1,actualz_pit_piFA%,actualz_pit_piSL%,actualz_pit_piCU%,actualz_pit_piCH%,actualz_pit_piFC%,appeared,mlb_link_id
0,13.0,2023-03-30T18:13:54,mlb-bp,LR,RP,21,Ian Kennedy,NaN,R,38.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,21.0,2023-03-30T18:13:54,il-pp,10IL,OF,23,Michael Brantley,NaN,L,35.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,16.0,2023-03-30T18:13:54,mlb-bp,LR,RP,60,Jesse Chavez,NaN,R,39.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,23.0,2023-03-30T18:13:54,il-rp,15IL,RP,35,Justin Wilson,NaN,L,35.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,7.0,2023-03-30T18:13:54,mlb-bn,Bench,OF,41,Jackie Bradley,NaN,L,32.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47,25.0,2024-04-14T19:15:51,aaa-rp,NaN,RP,74,Tyler Jay,NaN,L,30.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Y,664079.0
48,5.0,2024-04-14T22:51:18,aaa-sp,NaN,SP,--,Zak Kent,NaN,R,26.1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,687849.0
49,19.0,2024-04-14T19:15:51,mlb-bp,LR,RP,43,Anthony Molina,NaN,R,22.3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Y,683627.0
50,10.0,2024-04-14T22:51:18,mlb-bn,Bench,OF/INF,12,Max Schuemann,NaN,R,26.8,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Y,680474.0


In [6]:
changes_df[(changes_df['api_id'] == ['type']).all(1)]
changes_df[changes_df['apid_id'].isin(['type']).all(1)]

ValueError: ('Lengths must match to compare', (105,), (1,))